In [3]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers import apply_chunking_to_forward
from transformers.modeling_outputs import (
    BaseModelOutputWithPastAndCrossAttentions,
    BaseModelOutputWithPoolingAndCrossAttentions,
    MaskedLMOutput,
    TokenClassifierOutput,
    QuestionAnsweringModelOutput,
    SequenceClassifierOutput,
)
from transformers.modeling_utils import PreTrainedModel, find_pruneable_heads_and_indices, prune_linear_layer
from transformers.models.roberta.modeling_roberta import (
    RobertaIntermediate,
    RobertaLMHead,
    RobertaOutput,
    RobertaSelfOutput,
)
from transformers.utils import logging

from configuration_layoutlmv3 import LayoutLMv3Config
from timm.models.layers import to_2tuple


In [16]:
img_size=(14, 14)
max_len=1000
visual_bbox_x = torch.div(torch.arange(0, max_len * (img_size[1] + 1), max_len),
                                  img_size[1], rounding_mode='trunc')
visual_bbox_x,torch.arange(0, max_len * (img_size[1] + 1), max_len)
visual_bbox_y = torch.div(torch.arange(0, max_len * (img_size[0] + 1), max_len),
                                  img_size[0], rounding_mode='trunc')

In [20]:
visual_bbox = torch.stack(
            [
                visual_bbox_x[:-1].repeat(img_size[0], 1),
                visual_bbox_y[:-1].repeat(img_size[1], 1).transpose(0, 1),
                visual_bbox_x[1:].repeat(img_size[0], 1),
                visual_bbox_y[1:].repeat(img_size[1], 1).transpose(0, 1),
            ],
            dim=-1,
        ).view(-1, 4)
visual_bbox.shape,visual_bbox[0:3,:]

(torch.Size([196, 4]),
 tensor([[  0,   0,  71,  71],
         [ 71,   0, 142,  71],
         [142,   0, 214,  71]]))

In [22]:
cls_token_box = torch.tensor([[0 + 1, 0 + 1, max_len - 1, max_len - 1]])
visual_bbox = torch.cat([cls_token_box, visual_bbox], dim=0)
visual_bbox.shape

torch.Size([197, 4])

In [29]:
repeat=torch.randint(1,5,(3,))
repeat.shape,repeat.repeat(3,1).shape,repeat.repeat(3,2).shape,repeat.repeat(3,2,1).shape

(torch.Size([3]),
 torch.Size([3, 3]),
 torch.Size([3, 6]),
 torch.Size([3, 2, 3]))

In [34]:
repeat,repeat.expand(3,-1,-1)

RuntimeError: The expanded size of the tensor (-1) isn't allowed in a leading, non-existing dimension 1